This code is to automate sending the result of covid-19 tests using whatsapp.
The result being sended consists of two parts, first is a text message while the second one is a pdf file.
every work day, we have an excel file named Cumulative Data.xlsx which contains needed data; name, surename, caseID, phone number ..., where each row for one case.
This excel file is being used to generate pdfs files, a file for every case.

In [1]:
# config dirs 

date="26-05-2022"# This date is set daily; date of the day.

xlsxFileName="Cumulative Data.xlsx"
todayFolder=date[0:2]
month=date[3:5]
pdfFolder="Reports"

# mainDir="\\Main directory"
todayPath=mainDir+month+"\\"+todayFolder+"\\"
reportsPath=mainDir+month+"\\"+todayFolder+"\\"+pdfFolder+"\\"

# chech existance of pdf reports folder 
import os 
if os.path.exists(todayPath):
    print("path already exists")
else:
    os.mkdir(todayPath) 

In [2]:
# config xlsx dir

import xlrd
xlsxPath=todayPath+"\\"
xlsxloc=xlsxPath  +xlsxFileName
print(xlsxloc)

In [ ]:
# Read xlsx file and get needed values.

xlsxwb = xlrd.open_workbook(xlsxloc) 
contactlst=list()
sendedlst=list()
sheet = xlsxwb.sheet_by_name("Data")
for i in range(sheet.nrows):
    print(i)
    if i==0:
        continue
    else:
        if sheet.cell_value(i,28)<0: # Seq 
            continue
        else:
            sampleID=sheet.cell_value(i,0)
            sampleName=sheet.cell_value(i,2)
            sex=sheet.cell_value(i,8)
            PhoneNumbers=sheet.cell_value(i,22)
            res=sheet.cell_value(i,23)
            sampleSeq=sheet.cell_value(i,28)
            element=str(sampleID) + "_" +  str(sampleName) + "_" + str(int(PhoneNumbers)) + "_" + str(res)+"_"+str(sex)+"_"+str(sampleSeq)
            contactlst.append(element)
            sendedlst.append(str(int(sheet.cell_value(i,29))))
for indx,itm in enumerate(sendedlst):
                print(indx+1,":",itm)

In [4]:
#Update sheet to mark which result has been sent and which one has not.

import openpyxl
def updatesheet(filename,r,value):
    wb = openpyxl.load_workbook(filename)
    ws = wb.get_sheet_by_name('Data')
    x="AD"+r
    ws[x] = 1
    wb.save(filename)
    

def updatesheetbyonce(filename,lst):
    wb = openpyxl.load_workbook(filename)
    ws = wb.get_sheet_by_name("Data")
    for indx,elmnt in enumerate(lst):
        r=indx+2
        if elmnt==1:
            x="AD"+str(r)
            ws[x] = 1
    wb.save(filename)
    

In [ ]:
from selenium import webdriver

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep,time
import urllib.parse

path="\\ path of webdriver" # edge
driver=webdriver.Edge(executable_path= path)
Link = "https://web.whatsapp.com/"
NotSendedCounter=0
HasSendedCounter=0
def whatsapp_login():
    global wait, driver, Link
    wait = WebDriverWait(driver,10)
    driver.get(Link)
    sleep(10)
    print("SCAN YOUR QR CODE FOR WHATSAPP WEB IF DISPLAYED")
    driver.maximize_window()
    print("QR CODE SCANNED")

def send_message_and_attachment_to_unsaved_contact(number,msg,file_name):
    global Error
    Error=False
    t0=time()
    wrongNumber=False
    
    getURL=False
    pageload=False
    msgsent=False
    
    attachmentbutton=False
    sendattachbtn=False
    fileattach=False
    filefound=False
    
    
    print("\n==Start send_message==")
    params = {'phone': str(number), 'text': str(msg)}
    end = urllib.parse.urlencode(params)
    final_url = Link + 'send?' + end
    try:
        if len(str(number))<9:
            print("Phone Number digits is less than 9")
            print(time()-t0)
            Error=True
            return        
        driver.get(final_url)
        getURL=True
        print("URL Found successfully.")
        print(time()-t0)
    except:
        getURL=False
        print("URL was not Found successfully.")
        print(time()-t0)
    if getURL:
        getURL=False
        for retry1 in range(2):
            try:
                sleep(2)
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@title = "Menu"]')))
                sleep(2)
                pageload=True
                print("Page loaded successfully.")
                print(time()-t0)
                break
            except:
                pageload=False
                print("Page was not loaded successfully.")
                print(time()-t0)
                if retry1==1:###########
                    Error=True
                    print("Fail to Load page")
                    print(time()-t0)
                    break
    else:
        print("No Internet Connection")
        print(time()-t0)
        Error=True
        return
                    
    if pageload:
        pageload=False
        try:
            sleep(5)
            element=wait.until(EC.presence_of_element_located((By.XPATH,\
                    '//*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[2]/button')))
            element.click()
            sleep(8)
            msgsent=True
            wrongNumber=False
            print("Message has sent successfully.")
            print(time()-t0)
        except:
            Error=True
            msgsent=False
            wrongNumber=True
            print("Message has not sent successfully.")
            print(time()-t0)

        print("Msg Time for this case:", time()-t0)
        print(time()-t0)
    print(("==End send_message=="))
    print(time()-t0)
    
        #++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    
    if(msgsent):
        msgsent=False
        print("\n===Start send_attachment===")
        for retry2 in range(2):
            try:
                sleep(0.5)
                wait.until(EC.presence_of_element_located((By.XPATH, '//div[@title = "Attach"]'))).click()
                sleep(0.5)
                attachmentbutton=True
                print("Attachment button clicked successfully")
                print(time()-t0)
                break
            except:
                attachmentbutton=False
                print("Attachment button was not clicked successfully")
                print(time()-t0)
                if retry2==1:
                    print(file_name,"/Fail to click on attachment button.")
                    print(time()-t0)
                    Error=True
                    break
                    
        if attachmentbutton:
            attachmentbutton=False
            sleep(1)
            attachment = driver.find_element_by_xpath('//input[@accept="*"]')
            fileattach=True
            print("File button clicked successfully.")
            print(time()-t0)
        else:
            print("exp2")
            Error=True
        if fileattach:
            fileattach=False
            attachment.send_keys(file_name)
            filefound=True
            print("file found successfully")
            print(time()-t0)
            
        if filefound:
            filefound=False
            for retry3 in range(2):
                try:
                    sleep(0.5)
                    send = wait.until(EC.presence_of_element_located((By.XPATH, '//span[@data-icon="send"]')))
                    sendattachbtn=True
                    sleep(0.5)
                    print("Send attachment Button Found successfully")
                    print(time()-t0)
                    break
                except:
                    sendattachbtn=False
                    print("Send attachment Button was not Found successfully")
                    print(time()-t0)
                    if retry3==1:
                        Error=True
                        print(file_name,"/Fail to find send attachment button.")
                        print(time()-t0)
                        break
        else:
            print("exp3")
            Error=True
        if sendattachbtn:
            sendattachbtn=False
            sleep(0.5)
            send.click()
            print("Send attachment button clicked successfully")
            print(time()-t0)
            print("Time was taken for this case before sleep:", time()-t0)
            sleep(10)
        print("Time was taken for this case:", time()-t0)
        print("===End send_attachment===")
    else:
        print("Wrong Phone Number")
        Error=True          
#================================================================================================================                
    
if __name__ == "__main__":
    savecounter=0
#     notsendedlst=list()    
    print("Web Page Open")
    # Let us login and Scan
    whatsapp_login()
    print("was login")
    for iteration in range(5):
        print("*-*-*- Iteration",iteration,"*-*-*-")
        for indx, c in enumerate(contactlst):
            print("-----try to send to the case in record:",indx+1,"-----")
            if indx>1000:              #sequence of starting for multiple scripts if needed
                continue              
            if int(sendedlst[indx])==1:
                print("+++++ Already sended +++++")
                continue
            comp="المحترم/ة"
            prename="السيد/ة"
            fn=str(date[0:2])+str(date[3:5])+str(date[6:10])+" R "
            el=c.split("_")
            if el[4]=="ذكر":
                prename="السيد"
                comp="المحترم"
            elif el[4]=="أنثى":
                prename="السيدة"
                comp="المحترمة"
            we="مخبر الترصد الوبائي أرسل لك هذه الرسالة بشكل آلي بخصوص نتيجة تحليل المسحة الخاصة بكم."

            PositiveMessage ="{}\n\n{} {} {}:\n\nنودّ إخباركم بأن نتيجة الفحص المخبري الخاص بكم لتحري فيروس كورونا قد أظهر نتيجة إيجابية \n(تم اكتشاف الفيروس فيها). \nيرجى مراجعة أقرب مركز عزل لمرضى الكوفيد19 لمتابعة اجراءات تدبير الحالة. \nنتمنى لكم الشفاء العاجل.".format(we,prename, el[1],comp)
            NegativeMessage= "{}\n\n{} {} {}:\n\nنودّ إخباركم بأن نتيجة الفحص المخبري الخاص بكم لتحري فيروس كورونا قد أظهر نتيجة سلبية \n(لم يتم اكتشاف الفيروس فيها). \nيرجى الاستمرار بإجراءات الوقاية الشخصية.".format(we,prename, el[1],comp)
            RetestMessage=   "{}\n\n{} {} {}:\n\nنودّ إخباركم بأن نتيجة الفحص المخبري الخاص بكم لتحري فيروس كورونا قد أظهر نتيجة غير حاسمة \n(لذلك سوف يتم إعادة التحليل في المخبر). \nيرجى الاستمرار بإجراءات الوقاية الشخصية.".format(we,prename, el[1],comp)
            ResampleMessage= "{}\n\n{} {} {}:\n\nنودّ إخباركم بأن نتيجة الفحص المخبري الخاص بكم لتحري فيروس كورونا قد أظهر نتيجة غير حاسمة بعد تحليلها مرتين \n(وذلك يتطلب إعادة قطف العينة من حضرتكم). \nيرجى مراجعة أقرب مركز عزل لإعادة أخذ العينة مع الاستمرار باتباع الإجراءات الوقائية.".format(we,prename, el[1],comp)

            if el[3]=="Positive":
                message=PositiveMessage
            elif el[3]=="Negative":
                message=NegativeMessage
            elif el[3]=="إعادة تحليل":
                message=RetestMessage
            elif el[3]=="إعادة قطف":
                message=ResampleMessage
            else:
                message="---------"
            message=message+"\n  \n {}{}".format(fn ,int(float(el[5])))           
            if iteration!=2:
                send_message_and_attachment_to_unsaved_contact(el[2],message,reportsPath+"\\"+fn+ str(int(float(el[5])))+".pdf")
            elif iteration==2:
                send_message_and_attachment_to_unsaved_contact(el[2],message,reportsPath+"\\"+fn+ str(int(float(el[5])))+".pdf")
            else:
                continue
            if not Error:       
                HasSendedCounter+=1 
                sendedlst[indx]=1     
                print("\n\n ***Message and attachment have been sent successfully to: {} || {} || {} || {} || {}\n".format(el[0],el[1],el[2],el[3],str(int(float(el[5])))))
                savecounter+=1
                if savecounter==50:
                    savecounter=0
                    t2=time()
                    try:
                        updatesheetbyonce(todayPath+xlsxFileName,sendedlst)
                        print("Time for batch saving:", time()-t2)
                    except:
                        sleep(100)
                        updatesheetbyonce(todayPath+xlsxFileName,sendedlst)
                        print("Time for batch saving after sleeping:", time()-t2)
            else:
                NotSendedCounter+=1
                print("\n\n ###Fail to send to: {} || {} || {} || {} || {}\n".format(el[0],el[1],el[2],el[3],str(int(float(el[5])))))
    
    driver.close() # Close the Open tab
    
    for i,val in enumerate(sendedlst):
        print(i , ": ", val)
    updatesheetbyonce(todayPath+xlsxFileName,sendedlst)
    driver.quit()